<a href="https://colab.research.google.com/github/VelasquezE/ML4Sci_2025-II/blob/main/Module2/clasficadorTexto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Clasificador de texto**

# 0. Importar las librerías necesarias